In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid", palette="rainbow")
%matplotlib inline

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_651a44c6256d4289950e25263ac926ad = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_651a44c6256d4289950e25263ac926ad = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_651a44c6256d4289950e25263ac926ad = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='QaG9CCaAbuUWvmoq8UTBa8Oh9b4PM-yFULVsGptC0J94',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_651a44c6256d4289950e25263ac926ad)

body = client_651a44c6256d4289950e25263ac926ad.get_object(Bucket='waterqualitydrinkingwaterpotabili-donotdelete-pr-y19ucaazm8kmih',Key='water_potability.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2785 non-null   float64
 1   Hardness         3276 non-null   float64
 2   Solids           3276 non-null   float64
 3   Chloramines      3276 non-null   float64
 4   Sulfate          2495 non-null   float64
 5   Conductivity     3276 non-null   float64
 6   Organic_carbon   3276 non-null   float64
 7   Trihalomethanes  3114 non-null   float64
 8   Turbidity        3276 non-null   float64
 9   Potability       3276 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 256.1 KB


In [5]:
data.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

In [6]:
data.Potability.value_counts()

0    1998
1    1278
Name: Potability, dtype: int64

In [7]:
data= data.dropna()

In [8]:
data.Potability.value_counts()

0    1200
1     811
Name: Potability, dtype: int64

Not potable is much more potable(1200 > 811) so we need to balance the data to prevent bias.

In [9]:
notpotable  = data[data['Potability']==0]
potable = data[data['Potability']==1]  

from sklearn.utils import resample
df_minority_upsampled = resample(potable, replace = True, n_samples = 1200) 

from sklearn.utils import shuffle
data = pd.concat([notpotable, df_minority_upsampled])
data = shuffle(data) 

In [10]:
data.shape

(2400, 10)

In [11]:
data.Potability.value_counts()

0    1200
1    1200
Name: Potability, dtype: int64

In [12]:
data.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
2167,8.180755,178.079638,25623.867503,7.574082,391.918229,353.507014,19.749408,74.295305,3.845423,0
1664,6.336343,214.071982,14619.627261,6.584414,315.996793,370.795604,12.219411,83.668879,3.788803,0
2799,8.264091,216.551887,23556.934353,5.830685,333.500538,450.857491,8.667720,39.804179,3.410511,1
324,7.886591,213.052591,38852.029636,11.170789,281.700777,326.115350,10.385172,64.662501,4.589984,1
3268,6.702547,207.321086,17246.920347,7.708117,304.510230,329.266002,16.217303,28.878601,3.442983,1


In [13]:
data.to_csv('Waterqualitypreprocess.csv', index=False)

In [14]:
pd.read_csv('./Waterqualitypreprocess.csv')

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,8.180755,178.079638,25623.867503,7.574082,391.918229,353.507014,19.749408,74.295305,3.845423,0
1,6.336343,214.071982,14619.627261,6.584414,315.996793,370.795604,12.219411,83.668879,3.788803,0
2,8.264091,216.551887,23556.934353,5.830685,333.500538,450.857491,8.667720,39.804179,3.410511,1
3,7.886591,213.052591,38852.029636,11.170789,281.700777,326.115350,10.385172,64.662501,4.589984,1
4,6.702547,207.321086,17246.920347,7.708117,304.510230,329.266002,16.217303,28.878601,3.442983,1
...,...,...,...,...,...,...,...,...,...,...
2395,8.828094,196.562853,17699.610609,8.940380,379.914286,486.863267,17.517283,61.182844,3.793318,0
2396,7.397651,260.083385,26730.635334,8.076995,361.113199,376.482171,17.847827,101.466825,3.078479,1
2397,6.825468,187.051710,46140.126850,6.137757,352.702182,373.427235,12.606382,58.752201,3.027450,0
2398,6.512737,229.263861,22475.046873,12.580026,192.033592,479.501279,9.983462,66.668576,4.612964,1
